In [3]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import urllib.request
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
%matplotlib inline

In [4]:
'''NFL FASTR Documentation:
https://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'''

'NFL FASTR Documentation:\nhttps://www.rdocumentation.org/packages/nflscrapR/versions/1.8.1/topics/scrape_json_play_by_play'

In [6]:
'''Procure Data'''

URL = 'https://github.com/guga31bb/nflfastR-data/blob/master/'
Unzip = '.csv.gz?raw=True'

In [7]:
'''Pull data'''

def pull_PBP_data(YEAR):
    
    data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(YEAR) + Unzip,
                         compression='gzip', low_memory=False)
    
    return data

In [9]:
'''Data frames from last 3 years'''

data_2020 = pd.DataFrame(pull_PBP_data(2020))
data_2019 = pd.DataFrame(pull_PBP_data(2019))
data_2018 = pd.DataFrame(pull_PBP_data(2018))


In [10]:
# Neutral Situation

def Neutral_Situation_Call(DF):
    neutral_situation = DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8 & play_type != 'no_play'")
    return neutral_situation

neutral_situation = Neutral_Situation_Call(data_2020)

In [11]:
# Team List
Team_List = ['ARI',
'ATL',
'BAL',
'BUF',
'CAR',
'CHI',
'CIN',
'CLE',
'DAL',
'DEN',
'DET',
'GB',
'HOU',
'IND',
'JAX',
'KC',
'LA',
'LAC',
'LV',
'MIA',
'MIN',
'NE',
'NO',
'NYG',
'NYJ',
'PHI',
'PIT',
'SEA',
'SF',
'TB',
'TEN',
'WAS']





In [31]:
#CHOOSE TEAM

def Select_Team():
    
    team = input('Input Team Name:')
    
    team = team.upper()

    if team in Team_List:
        return team
    else:
        print('Please input a valid team')

    
    Team = team 



    return Team

Team = Select_Team()



In [46]:
## PLAYCALL FREQUENCY BY PLAY TYPE & DOWN

def PlayFreq_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.run_location, Play_filter.run_gap], columns= [Play_filter.down,Play_filter.shotgun] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.pass_location, Play_filter.pass_length], columns= [ Play_filter.down,Play_filter.shotgun] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    else:
        print("invalid play type")


    return round(Tab*100,1)

PlayFreq_ByTeam_Down(data_2020,'pass',Team)

<ipython-input-46-4d4cdc086a3c>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


down                                          1.0         2.0          3.0  \
shotgun                                         0     1     0     1      0   
posteam play_type pass_location pass_length                                  
ARI     pass      left          deep         16.7   3.8   0.0   5.1    0.0   
                                short        33.3  44.3   0.0  38.5    0.0   
                  middle        deep          0.0   4.7   0.0   0.9    0.0   
                                short         8.3  13.2  33.3  17.1    0.0   
                  right         deep          8.3   6.6  33.3   6.8    0.0   
                                short        33.3  27.4  33.3  31.6  100.0   

down                                               
shotgun                                         1  
posteam play_type pass_location pass_length        
ARI     pass      left          deep          9.1  
                                short        36.4  
                  middle        deep          6.8  
                                short        15.9  
                  right         deep         10.2  
                                short        21.6

In [40]:
## PLAYCALL TYPE FREQUENCY BY DOWN

def PlayFreqType_ByTeam_Down(DF, Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == 'pass') | (neutral_situation['play_type'] == 'run') ]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    
    Tab = pd.crosstab([Play_filter.posteam,  Play_filter.play_type], columns= [Play_filter.down,Play_filter.shotgun] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    

    return round(Tab*100,1)

PlayFreqType_ByTeam_Down(data_2020,Team)


<ipython-input-40-ed4a9f05ef4a>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


down                1.0         2.0         3.0      
shotgun               0     1     0     1     0     1
posteam play_type                                    
ARI     pass       37.1  42.9  33.3  63.2  33.3  76.2
        run        62.9  57.1  66.7  36.8  66.7  23.8

In [41]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['down', 'shotgun'], values = ['epa', 'yards_gained' ],fill_value = '-')
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['down', 'shotgun'], values = ['epa', 'yards_gained', 'air_yards'] ,fill_value = '-')
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down(data_2020,'pass',Team)

<ipython-input-41-efb900900ba0>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


air_yards                      \
down                                              1.0          2.0          
shotgun                                             0      1     0      1   
posteam play_type pass_location pass_length                                 
ARI     pass      left          deep             17.0  25.00     -  23.17   
                                short            2.25   1.60     -   4.58   
                  middle        deep                -  26.40     -  28.00   
                                short            15.0   5.29   4.0   3.55   
                  right         deep             27.0  26.43  21.0  27.38   
                                short           -0.75   4.31  10.0   3.84   

                                                              epa        \
down                                         3.0              1.0         
shotgun                                        0      1         0     1   
posteam play_type pass_location pass_length                               
ARI     pass      left          deep           -  24.12  0.926102  1.06   
                                short          -   4.75 -1.643913 -0.32   
                  middle        deep           -  28.67         - -0.34   
                                short          -   5.57  0.875424  0.37   
                  right         deep           -  28.78  -0.50009  0.69   
                                short        1.0   3.53  0.398399 -0.02   

                                                                             \
down                                              2.0             3.0         
shotgun                                             0     1         0     1   
posteam play_type pass_location pass_length                                   
ARI     pass      left          deep                -  1.14         - -0.34   
                                short               -  0.49         -  0.08   
                  middle        deep                - -0.79         - -1.39   
                                short        0.746432  0.69         -  0.24   
                  right         deep          2.38765 -0.68         - -0.33   
                                short       -0.864615 -0.04 -1.161387 -0.11   

                                            yards_gained                      \
down                                                 1.0          2.0          
shotgun                                                0      1     0      1   
posteam play_type pass_location pass_length                                    
ARI     pass      left          deep                16.0  19.25     -  17.00   
                                short               3.25   5.28     -   7.29   
                  middle        deep                   -  12.20     -   0.00   
                                short               16.0   7.29   6.0   8.25   
                  right         deep                 0.0  15.71  21.0   0.00   
                                short                5.0   6.86   0.0   4.27   

                                                        
down                                         3.0        
shotgun                                        0     1  
posteam play_type pass_location pass_length             
ARI     pass      left          deep           -  6.88  
                                short          -  5.38  
                  middle        deep           -  9.33  
                                short          -  7.36  
                  right         deep           -  6.89  
                                short        0.0  5.63

In [35]:
# EPA by Team, Play-type and Down -- HI LEVEL


def EPA_ByTeam_Down(DF, Team):

    Team_table = neutral_situation[neutral_situation['posteam']==Team]
    
    Table_2 = pd.pivot_table(Team_table, index= ['posteam','play_type', 'down'], columns = ['shotgun'], values = ['epa', 'yards_gained'], fill_value = '-')

    return round(pd.DataFrame(Table_2),2) 

EPA_ByTeam_Down(data_2020,Team)



epa       yards_gained      
shotgun                    0     1            0     1
posteam play_type down                               
ARI     pass      1.0  -0.27 -0.09         6.15  6.89
                  2.0   0.76  0.23         9.00  5.99
                  3.0  -1.16 -0.28         0.00  5.48
        run       1.0  -0.06 -0.06         3.68  4.37
                  2.0  -0.27 -0.02         2.50  4.17
                  3.0   0.83  0.31         2.00  6.21

In [42]:
## EPA BY DOWN & FORMATION


def PlayEPA_ByTeam_Down_Player(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['down', 'shotgun', 'rusher'], values = ['epa', 'yards_gained' ], aggfunc='mean', fill_value = '-')
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['down', 'shotgun'], values = ['epa', 'yards_gained', 'air_yards'], aggfunc='mean', fill_value = '-')
    else:
        print("invalid play type")


    return round(Tab,2)

PlayEPA_ByTeam_Down_Player(data_2020,'run',Team)

<ipython-input-42-7590cacf9968>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                      \
down                                         1.0                       
shotgun                                        0                       
rusher                                 C.Edmonds    C.Kirk D.Hopkins   
posteam play_type run_location run_gap                                 
ARI     run       left         end      -0.32207         -  0.993359   
                               guard   -0.269558         -         -   
                               tackle          -         -         -   
                  right        end             - -0.537777         -   
                               guard           -         -         -   
                               tackle  -0.789454         -         -   

                                                                              \
down                                                                           
shotgun                                                  1                     
rusher                                   K.Drake C.Edmonds K.Drake  K.Murray   
posteam play_type run_location run_gap                                         
ARI     run       left         end     -1.102878 -0.511438   -0.26  0.418723   
                               guard    -0.07905 -0.690545   -0.25 -0.144816   
                               tackle  -0.295192  -0.08845   -0.37  1.126694   
                  right        end      0.228174 -0.324561   -0.00   0.19556   
                               guard           -         -   -0.03         -   
                               tackle   0.083237         -    0.07  1.056841   

                                                                       ...  \
down                                         2.0                       ...   
shotgun                                        0                    1  ...   
rusher                                 C.Edmonds   K.Drake A.Isabella  ...   
posteam play_type run_location run_gap                                 ...   
ARI     run       left         end      -0.19253         -          -  ...   
                               guard           -         -          -  ...   
                               tackle          -         -          -  ...   
                  right        end             -         -  -1.803249  ...   
                               guard           - -1.128754          -  ...   
                               tackle          -         -          -  ...   

                                       yards_gained                    \
down                                            1.0       2.0           
shotgun                                           1         0           
rusher                                     K.Murray C.Edmonds K.Drake   
posteam play_type run_location run_gap                                  
ARI     run       left         end         8.333333       3.0       -   
                               guard            3.0         -       -   
                               tackle           8.0         -       -   
                  right        end         6.166667         -       -   
                               guard              -         -    -1.0   
                               tackle           8.5         -       -   

                                                                               \
down                                                                            
shotgun                                         1                               
rusher                                 A.Isabella C.Edmonds K.Drake  K.Murray   
posteam play_type run_location run_gap                                          
ARI     run       left         end              -       0.5    1.00       3.0   
                               guard            -         -    1.00         -   
                               tackle           -       4.0    3.83       4.5   
                  right        end           -6.0         - 

In [44]:
# PLAYER PLAY-TYPE FREQUENCY


def PlayFreqType_ByTeam_Down_Player(DF,PlayType,Team):
 
  


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[(neutral_situation['play_type'] == PlayType)]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.down,Play_filter.rusher], columns= [Play_filter.run_location,Play_filter.run_gap] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.down, Play_filter.receiver], columns= [Play_filter.pass_location,Play_filter.pass_length] ,values = Play_filter.play_id, aggfunc='count', normalize='index')
    else: 
        print("invalid play type")



    return round(Tab*100,1)

PlayFreqType_ByTeam_Down_Player(data_2020,'pass',Team)

<ipython-input-44-b581123d55ae>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


pass_location       left        middle         right       
pass_length         deep  short   deep  short   deep  short
down receiver                                              
1.0  A.Isabella     12.5   25.0   12.5   12.5    0.0   37.5
     C.Edmonds       0.0   77.8    0.0   11.1    0.0   11.1
     C.Kirk          0.0    8.3    0.0    0.0   25.0   66.7
     D.Arnold       10.0   10.0   10.0   20.0   30.0   20.0
     D.Daniels       0.0   33.3    0.0   16.7    0.0   50.0
     D.Hopkins       8.3   63.9    2.8   11.1    5.6    8.3
     K.Drake         0.0   58.3    0.0    8.3    0.0   33.3
     L.Fitzgerald    0.0    7.7   15.4   38.5    0.0   38.5
     M.Williams      0.0   50.0    0.0    0.0    0.0   50.0
     S.DeValve     100.0    0.0    0.0    0.0    0.0    0.0
     T.Sherfield     0.0  100.0    0.0    0.0    0.0    0.0
2.0  A.Isabella      0.0   37.5    0.0   25.0    0.0   37.5
     C.Edmonds       0.0   31.2    0.0   25.0    6.2   37.5
     C.Kirk          0.0   13.3    0.0   13.3   26.7   46.7
     D.Arnold       16.7   25.0    8.3    8.3    8.3   33.3
     D.Daniels       0.0    0.0    0.0    0.0  100.0    0.0
     D.Foster        0.0  100.0    0.0    0.0    0.0    0.0
     D.Hopkins      10.3   65.5    0.0   17.2    0.0    6.9
     J.Ward          0.0    0.0    0.0  100.0    0.0    0.0
     K.Drake         0.0   25.0    0.0   25.0    0.0   50.0
     K.Johnson       0.0   14.3    0.0   14.3   14.3   57.1
     L.Fitzgerald    0.0   42.9    0.0   21.4    7.1   28.6
     M.Williams     25.0   75.0    0.0    0.0    0.0    0.0
3.0  A.Isabella     14.3    0.0   14.3   14.3   42.9   14.3
     C.Edmonds       0.0   45.5    0.0   18.2    0.0   36.4
     C.Kirk          8.3    0.0    8.3   16.7   33.3   33.3
     D.Arnold        0.0    0.0   28.6   57.1    0.0   14.3
     D.Daniels       0.0    0.0    0.0    0.0    0.0  100.0
     D.Hopkins      14.3   60.7    3.6   10.7    0.0   10.7
     K.Drake         0.0    0.0    0.0    0.0    0.0  100.0
     K.Johnson       0.0   25.0   25.0    0.0   25.0   25.0
     L.Fitzgerald    7.7   53.8    0.0   15.4    0.0   23.1
     T.Sherfield     0.0  100.0    0.0    0.0    0.0    0.0

In [38]:
## EPA BY DOWN & FORMATION & Distance


def PlayEPA_ByTeam_Down_Distance(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8 & down > 1"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'run_location', 'run_gap'], columns = ['down', 'ydstogo'], values = ['epa', 'yards_gained' ],fill_value = '-')
    elif PlayType == 'pass':
        Tab = pd.pivot_table(Play_filter, index= ['posteam','play_type', 'pass_location', 'pass_length'], columns = ['shotgun', 'down'], values = ['epa', 'yards_gained', 'air_yards'],fill_value = '-')
    else:
        print("invalid play type")

    Final_df = pd.DataFrame(Tab)

    return round(Final_df,2)

PlayEPA_ByTeam_Down_Distance(data_2020,'run',Team)

<ipython-input-38-4543b362800b>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


epa                      \
down                                         2.0                       
ydstogo                                       1         2         3    
posteam play_type run_location run_gap                                 
ARI     run       left         end             -         -         -   
                               guard    0.908253         -         -   
                               tackle          - -0.084223 -0.628724   
                  right        end      0.634207         - -0.985645   
                               guard           -         -         -   
                               tackle  -0.308406         - -0.621373   

                                                                      \
down                                                                   
ydstogo                                       4         5         6    
posteam play_type run_location run_gap                                 
ARI     run       left         end     -1.078519  0.735897 -0.258004   
                               guard           -         -         -   
                               tackle   0.579134         - -0.040742   
                  right        end             - -0.823231         -   
                               guard   -0.044768 -0.413207         -   
                               tackle  -0.406245         -         -   

                                                                      \
down                                                                   
ydstogo                                       7         8         9    
posteam play_type run_location run_gap                                 
ARI     run       left         end             -         -         -   
                               guard           -         -         -   
                               tackle          -         -  1.491718   
                  right        end      1.156694 -0.846141 -1.803249   
                               guard           -         - -0.357301   
                               tackle          -  3.072247         -   

                                                  ... yards_gained             \
down                                              ...          3.0              
ydstogo                                       10  ...           4     5    6    
posteam play_type run_location run_gap            ...                           
ARI     run       left         end     -0.471834  ...          3.0     -    -   
                               guard           -  ...            -     -    -   
                               tackle  -0.661897  ...            -     -    -   
                  right        end     -0.525003  ...            -  13.0  1.0   
                               guard   -1.128754  ...            -     -    -   
                               tackle  -1.410267  ...            -     -    -   

                                                                              
down                                                                          
ydstogo                                  8     9     10   11   14    17   22  
posteam play_type run_location run_gap                                        
ARI     run       left         end        -  10.0  10.0    -    -  25.0  0.0  
                               guard      -     -     -  9.0    -     -    -  
                               tackle     -     -     -    -    -     -    -  
                  right        end      9.0     -   1.0    -  6.0     -    -  
                               guard      -     -     -    -    -     -    -  
                               tackle     -     -     -    -    -     -    -  

[6 rows x 54 columns]

In [39]:
## PLAYCALL FREQUENCY BY PLAY TYPE & DOWN & DIstance

def PlayFreq_ByTeam_Down(DF, PlayType, Team):
 
    PT = str(PlayType)


    neutral_situation = pd.DataFrame(DF.query("down < 4 & half_seconds_remaining > 120 & wp > .2 & wp <.8"))
    Play_filter = neutral_situation[neutral_situation['play_type'] == PT]
    Play_filter = Play_filter[neutral_situation['posteam'] == Team]

    if PlayType == 'run':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.run_location, Play_filter.run_gap], columns= [Play_filter.shotgun,Play_filter.down, Play_filter.ydstogo] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    elif PlayType == 'pass':
        Tab = pd.crosstab([Play_filter.posteam,Play_filter.play_type,Play_filter.pass_location, Play_filter.pass_length], columns= [ Play_filter.shotgun, Play_filter.down, Play_filter.ydstogo] ,values = Play_filter.play_id, aggfunc='count', normalize='columns')
    else:
        print("invalid play type")


    return round(Tab*100,1)

PlayFreq_ByTeam_Down(data_2020,'pass',Team)

<ipython-input-39-5023a5d5abdd>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Play_filter = Play_filter[neutral_situation['posteam'] == Team]


shotgun                                          0                            \
down                                           1.0         2.0           3.0   
ydstogo                                         5     10    6      10     1    
posteam play_type pass_location pass_length                                    
ARI     pass      left          deep         100.0   9.1   0.0    0.0    0.0   
                                short          0.0  36.4   0.0    0.0    0.0   
                  middle        deep           0.0   0.0   0.0    0.0    0.0   
                                short          0.0   9.1  50.0    0.0    0.0   
                  right         deep           0.0   9.1  50.0    0.0    0.0   
                                short          0.0  36.4   0.0  100.0  100.0   

shotgun                                         1                             \
down                                          1.0                              
ydstogo                                        4      6      9     10     15   
posteam play_type pass_location pass_length                                    
ARI     pass      left          deep          0.0    0.0    0.0   4.3    0.0   
                                short        33.3  100.0  100.0  42.6  100.0   
                  middle        deep          0.0    0.0    0.0   5.3    0.0   
                                short        33.3    0.0    0.0  13.8    0.0   
                  right         deep          0.0    0.0    0.0   7.4    0.0   
                                short        33.3    0.0    0.0  26.6    0.0   

shotgun                                      ...                          \
down                                         ...   3.0                     
ydstogo                                      ...    10    11    12    13   
posteam play_type pass_location pass_length  ...                           
ARI     pass      left          deep         ...  12.5   0.0   0.0   0.0   
                                short        ...   0.0  40.0  50.0   0.0   
                  middle        deep         ...  12.5   0.0  25.0  33.3   
                                short        ...  25.0  40.0   0.0  33.3   
                  right         deep         ...  12.5   0.0  25.0   0.0   
                                short        ...  37.5  20.0   0.0  33.3   

shotgun                                                                       \
down                                                                           
ydstogo                                        14    15     16     17     18   
posteam play_type pass_location pass_length                                    
ARI     pass      left          deep          0.0   0.0    0.0    0.0    0.0   
                                short        50.0  33.3  100.0    0.0  100.0   
                  middle        deep          0.0   0.0    0.0    0.0    0.0   
                                short         0.0   0.0    0.0  100.0    0.0   
                  right         deep         50.0  33.3    0.0    0.0    0.0   
                                short         0.0  33.3    0.0    0.0    0.0   

shotgun                                             
down                                                
ydstogo                                         20  
posteam play_type pass_location pass_length         
ARI     pass      left          deep           0.0  
                                short          0.0  
                  middle        deep           0.0  
                                short          0.0  
                  right         deep           0.0  
                                short        100.0  

[6 rows x 48 columns]